## DIA 109: Permitir que el Usuario Personalice la Frecuencia y Formato del Reporte (PDF o CSV)

Hoy vas a hacer que cada usuario elija con qué frecuencia (diaria, semanal, mensual) y en qué formato (PDF o CSV) desea recibir sus reportes por correo.

✅ ¿Qué aprenderás hoy?

🧑‍💼 Agregar campos personalizados a los usuarios.

📝 Crear un formulario de configuración.

🕒 Leer las preferencias para programar el envío.

📤 Generar PDF o CSV según la elección.

🧩 1. Extender el modelo User
Agrega dos nuevos campos a tu modelo:

python
Copiar
Editar
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)
    registros = db.relationship('Registro', backref='usuario', lazy=True)
    
    frecuencia_reporte = db.Column(db.String(10), default='semanal')  # diario, semanal, mensual
    formato_reporte = db.Column(db.String(10), default='pdf')         # pdf o csv
Recuerda hacer una migración si usas Flask-Migrate o ejecutar db.create_all() si es local y aún no hay datos.

🧩 2. Formulario para actualizar preferencias
Crea una ruta y una vista simple:

python
Copiar
Editar
@app.route('/configuracion-reporte', methods=['GET', 'POST'])
@login_required
def configuracion_reporte():
    if request.method == 'POST':
        current_user.frecuencia_reporte = request.form['frecuencia']
        current_user.formato_reporte = request.form['formato']
        db.session.commit()
        return redirect('/configuracion-reporte')
    
    return render_template('configuracion_reporte.html', user=current_user)
🧩 3. Template configuracion_reporte.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8">
  <title>Configuración de Reporte</title>
</head>
<body>
  <h2>📧 Configuración de Reporte</h2>
  <form method="POST">
    <label>Frecuencia:</label>
    <select name="frecuencia">
      <option value="diario" {% if user.frecuencia_reporte == 'diario' %}selected{% endif %}>Diario</option>
      <option value="semanal" {% if user.frecuencia_reporte == 'semanal' %}selected{% endif %}>Semanal</option>
      <option value="mensual" {% if user.frecuencia_reporte == 'mensual' %}selected{% endif %}>Mensual</option>
    </select>
    
    <br><br>

    <label>Formato:</label>
    <select name="formato">
      <option value="pdf" {% if user.formato_reporte == 'pdf' %}selected{% endif %}>PDF</option>
      <option value="csv" {% if user.formato_reporte == 'csv' %}selected{% endif %}>CSV</option>
    </select>

    <br><br>
    <button type="submit">Guardar</button>
  </form>
</body>
</html>
🧩 4. Lógica para enviar el formato correcto
En la función de envío automático:

python
Copiar
Editar
def generar_reporte(user, registros):
    if user.formato_reporte == 'csv':
        # Generar CSV
        data = [{
            "Fecha": r.fecha.strftime('%Y-%m-%d %H:%M:%S'),
            "F1": r.f1, "F2": r.f2, "F3": r.f3, "F4": r.f4,
            "Predicción": r.prediccion
        } for r in registros]
        df = pd.DataFrame(data)
        output = io.StringIO()
        df.to_csv(output, index=False)
        output.seek(0)
        return io.BytesIO(output.read().encode()), 'csv'
    else:
        # PDF por defecto
        return generar_pdf_con_grafico(user, registros), 'pdf'
Luego al enviar el correo:

python
Copiar
Editar
pdf_or_csv, tipo = generar_reporte(user, registros)
ext = 'pdf' if tipo == 'pdf' else 'csv'
mimetype = 'application/pdf' if tipo == 'pdf' else 'text/csv'

msg.add_attachment(pdf_or_csv.read(), maintype='application', subtype=ext,
                   filename=f'reporte_{user.username}.{ext}')
🧩 5. Separar tareas por frecuencia
En tu script programador con schedule:

python
Copiar
Editar
def enviar_por_frecuencia(frecuencia):
    with app.app_context():
        hoy = datetime.today()
        if frecuencia == "diario":
            desde = hoy - timedelta(days=1)
        elif frecuencia == "semanal":
            desde = hoy - timedelta(days=7)
        elif frecuencia == "mensual":
            desde = hoy - timedelta(days=30)

        usuarios = User.query.filter_by(frecuencia_reporte=frecuencia).all()
        for user in usuarios:
            registros = Registro.query.filter(
                Registro.user_id == user.id,
                Registro.fecha >= desde
            ).order_by(Registro.fecha).all()

            if registros:
                archivo, tipo = generar_reporte(user, registros)
                enviar_email_con_pdf(user, archivo, tipo)

schedule.every().day.at("08:00").do(lambda: enviar_por_frecuencia("diario"))
schedule.every().monday.at("08:00").do(lambda: enviar_por_frecuencia("semanal"))
schedule.every().day.at("08:05").do(lambda: enviar_por_frecuencia("mensual"))  # pue